## Otaku Castle Anime Recommender System
Data from Kaggle.com, collected from myanimelist.com

Source: https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset


### Dataset preprocessing and cleaning

In [1]:
# Import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Options
pd.set_option("display.max_rows", 100)

In [3]:
# Load dataframes
anime = pd.read_csv("anime-dataset-2023.csv")
scores = pd.read_csv("users-score-2023.csv")
users = pd.read_csv("users-details-2023.csv")

In [4]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   Name          24905 non-null  object
 2   English name  24905 non-null  object
 3   Other name    24905 non-null  object
 4   Score         24905 non-null  object
 5   Genres        24905 non-null  object
 6   Synopsis      24905 non-null  object
 7   Type          24905 non-null  object
 8   Episodes      24905 non-null  object
 9   Aired         24905 non-null  object
 10  Premiered     24905 non-null  object
 11  Status        24905 non-null  object
 12  Producers     24905 non-null  object
 13  Licensors     24905 non-null  object
 14  Studios       24905 non-null  object
 15  Source        24905 non-null  object
 16  Duration      24905 non-null  object
 17  Rating        24905 non-null  object
 18  Rank          24905 non-null  object
 19  Popu

In [5]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int64 
 1   Username     object
 2   anime_id     int64 
 3   Anime Title  object
 4   rating       int64 
dtypes: int64(3), object(2)
memory usage: 927.9+ MB


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731290 entries, 0 to 731289
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Mal ID            731290 non-null  int64  
 1   Username          731289 non-null  object 
 2   Gender            224383 non-null  object 
 3   Birthday          168068 non-null  object 
 4   Location          152805 non-null  object 
 5   Joined            731290 non-null  object 
 6   Days Watched      731282 non-null  float64
 7   Mean Score        731282 non-null  float64
 8   Watching          731282 non-null  float64
 9   Completed         731282 non-null  float64
 10  On Hold           731282 non-null  float64
 11  Dropped           731282 non-null  float64
 12  Plan to Watch     731282 non-null  float64
 13  Total Entries     731282 non-null  float64
 14  Rewatched         731282 non-null  float64
 15  Episodes Watched  731282 non-null  float64
dtypes: float64(10), int6

In [7]:
anime.head(3)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...


In [8]:
scores.head(3)

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5


In [9]:
users.head(3)

,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.6,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.8,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0


In [10]:
# Check all dataframes for missing data
anime.isna().sum()

anime_id        0
Name            0
English name    0
Other name      0
Score           0
Genres          0
Synopsis        0
Type            0
Episodes        0
Aired           0
Premiered       0
Status          0
Producers       0
Licensors       0
Studios         0
Source          0
Duration        0
Rating          0
Rank            0
Popularity      0
Favorites       0
Scored By       0
Members         0
Image URL       0
dtype: int64

In [11]:
scores.isna().sum()

user_id          0
Username       232
anime_id         0
Anime Title      0
rating           0
dtype: int64

In [12]:
users.isna().sum()

Mal ID                   0
Username                 1
Gender              506907
Birthday            563222
Location            578485
Joined                   0
Days Watched             8
Mean Score               8
Watching                 8
Completed                8
On Hold                  8
Dropped                  8
Plan to Watch            8
Total Entries            8
Rewatched                8
Episodes Watched         8
dtype: int64

In [13]:
# Users has missing data in 3 columns: Gender, Birthday, and Location. Roughly 2/3 of all records is missing this data, 
# so I will remove those columns from the dataset. 
users.drop(columns = ['Gender', 'Birthday', 'Location'], axis = 1, inplace = True)

In [14]:
# The "Joined" column is not helpful, and taking up space/processing time. I will remove it as well.
users.drop(columns = ['Joined'], axis = 1, inplace = True)

In [15]:
users.isna().sum()

Mal ID              0
Username            1
Days Watched        8
Mean Score          8
Watching            8
Completed           8
On Hold             8
Dropped             8
Plan to Watch       8
Total Entries       8
Rewatched           8
Episodes Watched    8
dtype: int64

In [16]:
# Check unique number of anime titles
anime.groupby('anime_id')['Name'].nunique().sum()

24905

In [17]:
# Checking scores in anime dataframe
anime['Score'].value_counts()

Score
UNKNOWN    9213
6.31         80
6.54         80
6.25         79
6.51         79
           ... 
3.21          1
3.29          1
1.85          1
3.69          1
4.07          1
Name: count, Length: 567, dtype: int64

Finding 9213 records with an unknown score will cause issues later. To fix this, I have choices. I could give these records a score of 1, delete those rows, or create a mean score from the other records. Before deciding, let's take a look at the data.

In [18]:
# Check for any anime with "Unknown" Score
anime.loc[anime['Score'] == "UNKNOWN"]

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
1578,1739,Shibawanko no Wa no Kokoro,UNKNOWN,しばわんこの和のこころ,UNKNOWN,UNKNOWN,Based on a japanese children`s book by Yoshie ...,TV,80.0,"Apr 5, 2006 to Mar 14, 2007",...,Three-d,Unknown,2 min per ep,G - All Ages,19660.0,13679,1,UNKNOWN,728,https://cdn.myanimelist.net/images/anime/11/70...
1699,1863,Silk Road Shounen Yuuto,UNKNOWN,シルクロード少年 ユート,UNKNOWN,"Adventure, Fantasy","When a boy Yuto visits Qinghai in China, he is...",TV,26.0,"Sep 16, 2006 to Mar 24, 2007",...,OLM,Unknown,20 min per ep,UNKNOWN,19912.0,13981,0,UNKNOWN,657,https://cdn.myanimelist.net/images/anime/1/186...
2476,2701,Susie-chan to Marvy,Little Susie and Marvy,スージーちゃんとマービー,UNKNOWN,Comedy,No description available for this anime.,TV,104.0,"Apr 5, 1999 to Feb 3, 2000",...,Xebec,Unknown,15 min per ep,UNKNOWN,12706.0,16212,0,UNKNOWN,374,https://cdn.myanimelist.net/images/anime/10/36...
2483,2708,Wankorobee,UNKNOWN,わんころべえ,UNKNOWN,"Comedy, Fantasy",No description available for this anime.,TV,26.0,"Oct 6, 1996 to Mar 30, 1997",...,"Ajia-do, TMS Entertainment",Original,15 min per ep,G - All Ages,13327.0,16545,0,UNKNOWN,348,https://cdn.myanimelist.net/images/anime/11/36...
2508,2735,Mugen Senki Portriss,UNKNOWN,無限戦記ポトリス,UNKNOWN,"Action, Sci-Fi","In a raving world, legendary knights stood up....",TV,52.0,"Apr 5, 2003 to Mar 27, 2004",...,"Sunrise, Dongwoo A&E",Unknown,25 min per ep,PG - Children,18320.0,14460,1,UNKNOWN,567,https://cdn.myanimelist.net/images/anime/1627/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",...,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,UNKNOWN,24723,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",...,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1931/...


Since very few members have watched/favorited these shows with UNKNOWN as a score, I have decided to give them a 0.5 score. This keeps them from being eliminated from the dataset, and maybe allows someone in the future to discover a hidden gem.

In [19]:
anime['Score'] = anime['Score'].replace('UNKNOWN', '0.5')

In [20]:
# Rerun check to test
anime.loc[anime['Score'] == "UNKNOWN"]

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL


In [21]:
# Perform same on Rank and Popularity to replace 'UNKNOWN'
anime['Rank'] = anime['Rank'].replace('UNKNOWN', '0.0')
anime['Popularity'] = anime['Popularity'].replace('UNKNOWN', '0')

The data is clean, and preprocessed. 

Now to dig deeper into the data and possibly create some visualizations that will help in building the Recommender system.

### Data Exploration

This section is not necessary for the model to function, but it will help the creation process to see the data explored both in text and visually.

In [22]:
# Look over scores info
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int64 
 1   Username     object
 2   anime_id     int64 
 3   Anime Title  object
 4   rating       int64 
dtypes: int64(3), object(2)
memory usage: 927.9+ MB


In [23]:
scores.head(1)

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9


In [24]:
# Look over users info
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731290 entries, 0 to 731289
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Mal ID            731290 non-null  int64  
 1   Username          731289 non-null  object 
 2   Days Watched      731282 non-null  float64
 3   Mean Score        731282 non-null  float64
 4   Watching          731282 non-null  float64
 5   Completed         731282 non-null  float64
 6   On Hold           731282 non-null  float64
 7   Dropped           731282 non-null  float64
 8   Plan to Watch     731282 non-null  float64
 9   Total Entries     731282 non-null  float64
 10  Rewatched         731282 non-null  float64
 11  Episodes Watched  731282 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 67.0+ MB


In [25]:
users.head(1)

,Mal ID,Username,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0


In [26]:
# Check unique number of users
users.groupby('Mal ID')['Username'].nunique().sum()

731289

In [27]:
# Check unique number of scores
scores.groupby('user_id')['Username'].nunique().sum()

270032

In [28]:
# Does the Mal ID in 'users' match 'user_id' in scores?
users['Mal ID'].isin(scores['user_id']).value_counts()

Mal ID
False    467221
True     264069
Name: count, dtype: int64

In [29]:
# Does the 'Username' in users match 'Username' in scores?
users['Username'].isin(scores['Username']).value_counts()

Username
False    467221
True     264069
Name: count, dtype: int64

We have the same number of Usernames and user_ids betweeen the 2 dataframes. Not everyone gives a score to the anime they watch, but almost every score has a user tied to it. This is good info, and will help later.

In [30]:
# Number of anime titles by type
print(anime.groupby('Type').size())

Type
Movie      4381
Music      2686
ONA        3533
OVA        4076
Special    2558
TV         7597
UNKNOWN      74
dtype: int64
